In [6]:
!pip install backoff
! pip install SPARQLWrapper
! pip install transformers

import json
import random
import os
import pandas as pd
from tqdm.asyncio import tqdm
import re
import aiohttp
import asyncio
import backoff
import nest_asyncio
from SPARQLWrapper import SPARQLWrapper, JSON
from transformers import pipeline



RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
module 'tensorflow' has no attribute 'data'

In [3]:
####################
# READ THE JSON
#####################

json_file_path = "C:/ALESSANDRO/Università/MAGISTRALE/SINTEF_thesis/lamAPI/data/Round4_sorted_mentions.json"

# Load the JSON file
with open(json_file_path, "r") as file:
    R4_sorted_mentions = json.load(file)

In [4]:
## Sample extraction
# SPLIT OVER THE QUARTILES

n = len(R4_sorted_mentions)
q1_idx = n // 4
q2_idx = n // 2
q3_idx = 3 * n // 4

# Step 3: Split the list into quartiles
q1 = R4_sorted_mentions[:q1_idx]
q2 = R4_sorted_mentions[q1_idx:q2_idx]
q3 = R4_sorted_mentions[q2_idx:q3_idx]
q4 = R4_sorted_mentions[q3_idx:]

sample_size = int(len(R4_sorted_mentions)/40)
R4_sample_keys = []
R4_sample_keys = R4_sample_keys + random.sample(q1, sample_size)
R4_sample_keys = R4_sample_keys + random.sample(q2, sample_size)
R4_sample_keys = R4_sample_keys + random.sample(q3, sample_size)
R4_sample_keys = R4_sample_keys + random.sample(q4, sample_size)

q_ids = {item[1]['name']: item[1]['id'] for item in R4_sample_keys}
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

NameError: name 'pipeline' is not defined

In [33]:
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

mentions = {}
labels = ["ORGANISATION", "LOCATION", "PERSON"]
total_iterations = len(q_ids)
progress_bar = tqdm(total=total_iterations, desc="Processing")

def process_item(key, value):
    classifier_output = classifier(key, labels)
    max_score_index = classifier_output['scores'].index(max(classifier_output['scores']))
    highest_label = classifier_output['labels'][max_score_index]

    if classifier_output['scores'][max_score_index] > 0.5:
        return key, (value, highest_label)
    else:
        return key, (value, "OTHERS")

# Control the number of parallel tasks here
max_workers = 4  # Set the number of worker threads

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_item, k, v) for k, v in q_ids.items()]
    for future in tqdm(as_completed(futures), total=len(q_ids), desc="Processing"):
        try:
            key, value = future.result()
            mentions[key] = value
            progress_bar.update(1)
        except Exception as exc:
            print(f"Error processing {key}: {exc}")

# Close the tqdm progress bar
progress_bar.close()


Processing:   0%|          | 0/45638 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-33-11ca7f085057>", line 24, in <cell line: 22>
    for future in tqdm(as_completed(futures), total=len(q_ids), desc="Processing"):
NameError: name 'as_completed' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-33-11ca7f085057>", line 22, in <cell line: 22>
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 649, in __exit__
    self.shutdown(wait=True)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 235, in shutdown
    t.join()
  File "/usr/lib/python3.10/threading.py", line 1096, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.10/threading.py", line 1116, in _wait_for


KeyboardInterrupt



In [31]:
mentions = {}

# Extract labels from the mapping dictionary
labels = ["ORGANISATION", "LOCATION", "PERSON"]
total_iterations = len(q_ids)
progress_bar = tqdm(total=total_iterations, desc="Processing")

for k, v in q_ids.items():
    classifier_output = classifier(k, labels)

    max_score_index = classifier_output['scores'].index(max(classifier_output['scores']))
    highest_label = classifier_output['labels'][max_score_index]

    if classifier_output['scores'][classifier_output['labels'] == highest_label] > 0.5:
      mentions[k] = (v, highest_label)
    else:
      mentions[k] = (v, "OTHERS")

    print(f"{k} = {mentions[k]} with score: {classifier_output['scores'][classifier_output['labels'] == highest_label]}")
    progress_bar.update(1)

# Close the tqdm progress bar
progress_bar.close()

Processing:   0%|          | 1/45638 [00:04<53:34:16,  4.23s/it]

formula könig = ('https://www.wikidata.org/entity/Q656937', 'OTHERS') with score: 0.42270973324775696


Processing:   0%|          | 2/45638 [00:07<43:55:47,  3.47s/it]

mind and life institute = ('https://www.wikidata.org/entity/Q462154', 'ORGANISATION') with score: 0.8899603486061096


Processing:   0%|          | 3/45638 [00:09<34:48:35,  2.75s/it]

...

Processing:   0%|          | 170/45638 [04:57<22:00:53,  1.74s/it]

model = ('https://www.wikidata.org/entity/Q70536', 'OTHERS') with score: 0.4726133346557617


Processing:   0%|          | 171/45638 [04:59<22:17:48,  1.77s/it]

the pacific = ('https://www.wikidata.org/entity/Q598499', 'LOCATION') with score: 0.7851206064224243


KeyboardInterrupt: 

In [ ]:
url = 'https://lamapi.hel.sintef.cloud/lookup/entity-retrieval'

# Backoff decorator for handling retries with exponential backoff
@backoff.on_exception(
    backoff.expo,
    (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError, asyncio.TimeoutError),
    max_tries=5,
    max_time=300
)
async def fetch(session, url, params, headers, semaphore):
    async with semaphore:
        async with session.get(url, params=params, headers=headers, timeout=30) as response:
            try:
                response.raise_for_status()  # Raises an exception for 4XX/5XX status codes
                return await response.json()
            except Exception as e:
                return []

async def process_item(session, name, value, url, headers, semaphore, pbar):
    params = {
        'name': name,
        'token': 'lamapi_demo_2023',
        'kg': 'wikidata',
        'limit': 1000,
        'query': f'''
        {{
            "query": {{
                "bool": {{
                    "must": [
                        {{
                            "match": {{
                                "name": {{
                                    "query": "{name}",
                                    "boost": 2.0
                                }}
                            }}
                        }},
                        {{
                            "term": {{
                                "NERtype": "{value[1]}"
                            }}
                        }}
                    ]
                }}
            }}
        }}
        '''
    }

    try:
        data = await fetch(session, url, params, headers, semaphore)
    except ClientResponseError as e:
        if e.status == 404:
            print(f"404 Error: Resource not found for '{name}'")
            pbar.update(1)  # No need to await here
            return 0, 0
        else:
            raise  # Re-raise the exception for other status codes

    num_result = len(data) if data else 0

    if data:
        for item in data:
            GT_id_match = re.search(r'Q(\d+)$', value[0])
            if GT_id_match:
                GT_id = GT_id_match[0]
                if GT_id == item.get('id'):
                    pbar.update(1)  # No need to await here
                   # print(f"{el}-->t{item}")
                    #print("__________________________")
                    pos_score = item.get('pos_score', 0)
                    if pos_score:
                        mrr_increment = (num_result - (pos_score * num_result)) / num_result
                    else:
                        mrr_increment = 1 / num_result  # Assume worst case for MRR if pos_score is 0
                    return mrr_increment, 1


    return 0, 0

async def main(mentions, url, pbar):
    string_name_list = mentions
    headers = {'accept': 'application/json'}
    semaphore = asyncio.Semaphore(50)  # Limit to 50 concurrent requests
    m_mrr = 0
    cont_el = 0
    async with aiohttp.ClientSession() as session:
        tasks = []
        for name, type in string_name_list.items():
            tasks.append(process_item(session, name, type, url, headers, semaphore, pbar))

        results = await asyncio.gather(*tasks)

        for mrr_increment, count in results:
            m_mrr += mrr_increment
            cont_el += count

        pbar.close()  # No need to await here

    print(f"Coverage of R1: {cont_el / len(mentions)}")
    print(f"Measure Reciprocal Rank of R1: {m_mrr / len(mentions)}")

# Check if there's already a running event loop
if __name__ == "__main__":
    nest_asyncio.apply()  # Apply nest_asyncio
    try:
        pbar = tqdm(total=len(mentions))
        asyncio.run(main(mentions, url, pbar))
    except RuntimeError:  # For environments like Jupyter
        loop = asyncio.get_event_loop()
        loop.run_until_complete(main(mentions, url, pbar))


 54%|█████▎    | 8175/15224 [09:57<08:34, 13.69it/s]  


Coverage of R1: 0.5369810825013137
Measure Reciprocal Rank of R1: 0.5103510246977966
